Feb 21

The OWER classifier does learn something - training loss decreases
 as does validation until the 5th epoch or so - but
validation loss decreases only slightly, and the results on
"Barack Obama is a married, male, American actor" look bad.

It is worth turning the weights of the loss function but still the
resulting classes for Obama differ randomly from run to run. Also,
plotting the class/word attentions shows that those converge, but
the observable correlations are not as expected.

In the following a minimal example shall be constructed to proof
the concept behind the classifier. An entity with a few short
sentences from a small vocabulary shall be forwarded through a
pre-trained OWER classifier whose class embeddings perfectly match
the sentences so that the correct classes should be predicted.

Example scenario:

```
Input sentences: "married married", "male male", "american american"
Expected classes: married=True, male=True, american=True, actor=?
```

# Set up helpers

In [ ]:
from IPython.lib.pretty import pretty

def log_tensor(tensor, title, labels, vmin=None, vmax=None):
    pass

%run util.ipynb

# 1 Input data

Given an input such as `raw_input` in the following, pre-processing
should yield a `sents_batch` and a `classes_batch`, each of size 1.

In [ ]:
data = [
    { 'ent': 123, 'classes': [1, 1, 1], 'sents': ['married married married', 'male male male', 'American American American'] },
    { 'ent': 123, 'classes': [1, 1, 1], 'sents': ['married married married', 'male male male', 'American American American'] },
]

# 2 Pre-processing

## 2.1 Build vocabulary

In [ ]:
from collections import Counter
from torchtext.vocab import Vocab

def tokenize(text):
    return text.split()

words = [word for ent in data for sent in ent['sents'] for word in tokenize(sent)]
vocab = Vocab(Counter(words))

print(pretty(vocab.stoi))

## 2.2 Transform data

Map words to tokens and create tensors.

In [ ]:
import torch
from torch import tensor

sents_batch = tensor([[[vocab[word] for word in tokenize(sent)] for sent in ent['sents']] for ent in data])
classes_batch = torch.tensor([ent['classes'] for ent in data])

assert len(sents_batch) == len(classes_batch)

batch_size = len(sents_batch)
sent_count = 3
sent_len = 3

ent_labels = [f'ent {i}' for i in range(batch_size)]
sent_labels = [f'sent {i}' for i in range(sent_count)]
tok_labels = [f'tok {i}' for i in range(sent_len)]

log_tensor(sents_batch, 'sents_batch', [ent_labels, sent_labels, tok_labels])

# 3 Prepare classifier

## 3.1 Prepare EmbeddingBag

Create and prepare an `EmbeddingBag` so that each token has a unique,
easily memorable embedding.

In [ ]:
from torch.nn import EmbeddingBag
from torch import tensor

vocab_size = len(vocab)
assert vocab_size == 5

emb_size = 4

embedding_bag = EmbeddingBag(num_embeddings=vocab_size, embedding_dim=emb_size)

embedding_bag.weight.data = tensor([
    [0., 0., 0., 0.],
    [1., 0., 0., 0.],
    [0., 1., 0., 0.],
    [0., 0., 1., 0.],
    [0., 0., 0., 1.]
])

old_embedding_bag_weight_data = embedding_bag.weight.data.detach().clone()

log_output = 1

word_labels = ['<unk>', '<pad>', 'married', 'male', 'American']
emb_labels = [f'emb {i}' for i in range(emb_size)]

if log_output:
    log_tensor(embedding_bag.weight, 'embedding_bag.weight', [word_labels, emb_labels])

## 3.2 Prepare class embeddings

Create and prepare the class embeddings so that there is one class embedding
per non-special token that perfectly matches that token's embedding.

In [ ]:
class_embs = tensor([
    [0., 1., 0., 0.],
    [0., 0., 1., 0.],
    [0., 0., 0., 1.]
], requires_grad=True)

old_class_embs = class_embs.detach().clone()

log_output = 1

class_labels = ['married', 'male', 'American']

if log_output:
    log_tensor(class_embs, 'class_embs', [class_labels, emb_labels])

## 3.3 Prepare linear layer

Create and prepare the linear layer so that it completely relies on
the first class' sentence mix to predict the first class, the second
class' sentence mix to predict the second class, etc.

In [ ]:
from torch.nn import Linear

class_count = len(class_embs)
assert class_count == 3

linear = Linear(class_count * emb_size, class_count)

linear.weight.data = tensor([
    [1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.]
])

linear.bias.data = tensor([0., 0., 0.])

old_linear_weight_data = linear.weight.data.detach().clone()
old_linear_bias_data = linear.bias.data.detach().clone()

log_output = 1

mix_emb_labels = [f'mix {i} / emb {j}' for i in range(class_count) for j in range(emb_size)]

if log_output:
    log_tensor(linear.weight.data, 'linear.weight.data', [class_labels, mix_emb_labels])
    log_tensor(linear.bias.data, 'linear.bias.data', [class_labels])

# 4 Forward

## 4.1 Embed sentences

In [ ]:
#
# Flatten batch
#
# < sents_batch  (batch_size, sent_count, sent_len)
# > flat_sents   (batch_size * sent_count, sent_len)
#

flat_sents = sents_batch.reshape(batch_size * sent_count, sent_len)

log_input = 0
log_output = 0

ent_sent_labels = [f'ent {i} / sent {j}' for i in range(batch_size) for j in range(sent_count)]

if log_input:
    log_tensor(sents_batch, 'sents_batch', [ent_labels, sent_labels, tok_labels])

if log_output:
    log_tensor(flat_sents, 'flat_sents', [ent_sent_labels, tok_labels])

In [ ]:
#
# Embed sentences
#
# < embedding_bag.weight  (vocab_size, emb_size)
# < flat_sents            (batch_size * sent_count, sent_len)
# > flat_sent_embs        (batch_size * sent_count, emb_size)
#

flat_sent_embs = embedding_bag(flat_sents)

log_input = 0
log_output = 0

if log_input:
    log_tensor(embedding_bag.weight, 'embedding_bag.weight', [word_labels, emb_labels])
    log_tensor(flat_sents, 'flat_sents', [ent_sent_labels, tok_labels])

if log_output:
    log_tensor(flat_sent_embs, 'flat_sent_embs', [ent_sent_labels, emb_labels])

In [ ]:
#
# Restore batch
#
# < flat_sent_embs   (batch_size * sent_count, emb_size)
# > sent_embs_batch  (batch_size, sent_count, emb_size)
#

sent_embs_batch = flat_sent_embs.reshape(batch_size, sent_count, emb_size)

log_input = 0
log_output = 1

if log_input:
    log_tensor(flat_sent_embs, 'flat_sent_embs', [ent_sent_labels, emb_labels])

if log_output:
    log_tensor(sent_embs_batch, 'sent_embs_batch', [ent_labels, sent_labels, emb_labels])

## 4.2 Calc attentions

In [ ]:
#
# Expand class embeddings for bmm()
#
# < class_embs        (class_count, emb_size)
# > class_embs_batch  (batch_size, class_count, emb_size)
#

class_embs_batch = class_embs.expand(batch_size, class_count, emb_size)

log_input = 0
log_output = 0

if log_input:
    log_tensor(class_embs, 'class_embs', [class_labels, emb_labels])

if log_output:
    log_tensor(class_embs_batch, 'class_embs_batch', [ent_labels, class_labels, emb_labels])

In [ ]:
#
# Multiply each class with each sentence
#
# < class_embs_batch    (batch_size, class_count, emb_size)
# < sent_embs_batch     (batch_size, sent_count, emb_size)
# > atts_batch          (batch_size, class_count, sent_count)
#

atts_batch = torch.bmm(class_embs_batch, sent_embs_batch.transpose(1, 2))

log_input = 0
log_output = 0

if log_input:
    log_tensor(class_embs_batch, 'class_embs_batch', [ent_labels, class_labels, emb_labels])
    log_tensor(sent_embs_batch, 'sent_embs_batch', [ent_labels, sent_labels, emb_labels])

if log_output:
    log_tensor(atts_batch, 'atts_batch', [ent_labels, class_labels, sent_labels])

In [ ]:
#
# Apply softmax over sentences
#
# < atts_batch      (batch_size, class_count, sent_count)
# > softs_batch     (batch_size, class_count, sent_count)
#

from torch.nn import Softmax

softs_batch = Softmax(dim=-1)(atts_batch)

log_input = 0
log_output = 1

if log_input:
    log_tensor(atts_batch, 'atts_batch', [ent_labels, class_labels, sent_labels])

if log_output:
    log_tensor(softs_batch, 'softs_batch', [ent_labels, class_labels, sent_labels])

## 4.3 Weight and mix sentences

In [ ]:
#
# Repeat each batch slice class_count times
#
# < sent_embs_batch     (batch_size, sent_count, emb_size)
# > expaned_batch       (batch_size, class_count, sent_count, emb_size)
#

expaned_batch = sent_embs_batch.unsqueeze(1).expand(-1, class_count, -1, -1)

log_input = 0
log_output = 0

if log_input:
    log_tensor(sent_embs_batch, 'sent_embs_batch', [ent_labels, sent_labels, emb_labels])

if log_output:
    log_tensor(expaned_batch, 'expaned_batch', [ent_labels, class_labels, sent_labels, emb_labels])

In [ ]:
#
# Flatten sentences for bmm()
#
# < expaned_batch   (batch_size, class_count, sent_count, emb_size)
# > flat_expanded   (batch_size * class_count, sent_count, emb_size)
#

flat_expanded = expaned_batch.reshape(-1, sent_count, emb_size)

log_input = 0
log_output = 0

ent_class_labels = [f'ent {i} / class {j}' for i in range(batch_size) for j in range(class_count)]

if log_input:
    log_tensor(expaned_batch, 'expaned_batch', [ent_labels, class_labels, sent_labels, emb_labels])

if log_output:
    log_tensor(flat_expanded, 'flat_expanded', [ent_class_labels, sent_labels, emb_labels])

In [ ]:
#
# Flatten attentions for bmm()
#
# < softs_batch     (batch_size, class_count, sent_count)
# > flat_softs      (batch_size * class_count, sent_count, 1)
#

flat_softs = softs_batch.reshape(batch_size * class_count, sent_count).unsqueeze(-1)

log_input = 0
log_output = 0

if log_input:
    log_tensor(softs_batch, 'softs_batch', [ent_labels, class_labels, sent_labels])

if log_output:
    log_tensor(flat_softs, 'flat_softs', [ent_class_labels, sent_labels, ['']])

In [ ]:
#
# Multiply each sentence with each attention
#
# < flat_expanded   (batch_size * class_count, sent_count, emb_size)
# < flat_softs      (batch_size * class_count, sent_count, 1)
# > flat_weighted   (batch_size * class_count, emb_size)
#

flat_weighted = torch.bmm(flat_expanded.transpose(1, 2), flat_softs).squeeze(-1)

log_input = 0
log_output = 0

if log_input:
    log_tensor(flat_expanded, 'flat_expanded', [ent_class_labels, sent_labels, emb_labels])
    log_tensor(flat_softs, 'flat_softs', [ent_class_labels, sent_labels, ['']])

if log_output:
    log_tensor(flat_weighted, 'flat_weighted', [ent_class_labels, emb_labels])

In [ ]:
#
# Restore batch
#
# < flat_weighted   (batch_size * class_count, emb_size)
# > weighted_batch  (batch_size, class_count, emb_size)
#

weighted_batch = flat_weighted.reshape(batch_size, class_count, emb_size)

log_input = 0
log_output = 1

if log_input:
    log_tensor(flat_weighted, 'flat_weighted', [ent_class_labels, emb_labels])

if log_output:
    log_tensor(weighted_batch, 'weighted_batch', [ent_labels, class_labels, emb_labels])

## 4.4 Linear layer

In [ ]:
#
# Concatenate mixes
#
# < weighted_batch  (batch_size, class_count, emb_size)
# > concat_mixes_batch  (batch_size, class_count * emb_size)
#

concat_mixes_batch = weighted_batch.reshape(batch_size, class_count * emb_size)

log_input = 0
log_output = 1

mix_emb_labels = [f'mix {i} / emb {j}' for i in range(class_count) for j in range(emb_size)]

if log_input:
    log_tensor(weighted_batch, 'weighted_batch', [ent_labels, class_labels, emb_labels])

if log_output:
    log_tensor(concat_mixes_batch, 'concat_mixes_batch', [ent_labels, mix_emb_labels])

In [ ]:
#
# Push concatenated mixes through linear layer
#
# < concat_mixes_batch  (batch_size, class_count * emb_size)
# > logits_batch        (batch_size, class_count)
#

logits_batch = linear(concat_mixes_batch)

log_input = 0
log_output = 1

if log_input:
    log_tensor(concat_mixes_batch, 'concat_mixes_batch', [ent_labels, mix_emb_labels])

if log_output:
    log_tensor(logits_batch, 'logits_batch', [ent_labels, class_labels], vmin=-1, vmax=1)

## 4.5 Loss

In [ ]:
#
# Push concatenated mixes through linear layer
#
# < concat_mixes_batch  (batch_size, class_count * emb_size)
# > logits_batch        (batch_size, class_count)
#

from torch.nn import BCEWithLogitsLoss

criterion = BCEWithLogitsLoss(pos_weight=torch.tensor([80] * class_count))
# criterion = torch.nn.MSELoss()

loss = criterion(logits_batch, classes_batch.float())

log_input = 0
log_output = 1

if log_input:
    log_tensor(logits_batch, 'logits_batch', [ent_labels, class_labels], vmin=-1, vmax=1)
    log_tensor(classes_batch, 'classes_batch', [ent_labels, class_labels], vmin=-1, vmax=1)

log_tensor(loss, 'loss', [])

# 5 Backward

In [ ]:
from torch.optim import Adam

optimizer = Adam([embedding_bag.weight, class_embs, linear.weight, linear.bias], lr=0.1)

In [ ]:
optimizer.zero_grad()
loss.backward()

In [ ]:
optimizer.step()

vmin = min(torch.min(old_embedding_bag_weight_data), torch.min(embedding_bag.weight.data))
vmax = max(torch.max(old_embedding_bag_weight_data), torch.max(embedding_bag.weight.data))

log_tensor(old_embedding_bag_weight_data, 'old_embedding_bag_weight_data', [word_labels, emb_labels], vmin=vmin, vmax=vmax)
log_tensor(embedding_bag.weight.data.detach(), 'embedding_bag.weight.data', [word_labels, emb_labels], vmin=vmin, vmax=vmax)

vmin = min(torch.min(old_class_embs), torch.min(class_embs))
vmax = max(torch.max(old_class_embs), torch.max(class_embs))

log_tensor(old_class_embs, 'old_class_embs', [class_labels, emb_labels], vmin=vmin, vmax=vmax)
log_tensor(class_embs.detach(), 'class_embs', [class_labels, emb_labels], vmin=vmin, vmax=vmax)

vmin = min(torch.min(old_linear_weight_data), torch.min(linear.weight.data))
vmax = max(torch.max(old_linear_weight_data), torch.max(linear.weight.data))

log_tensor(old_linear_weight_data, 'old_linear_weight_data', [class_labels, mix_emb_labels], vmin=vmin, vmax=vmax)
log_tensor(linear.weight.data.detach(), 'linear.weight.data', [class_labels, mix_emb_labels], vmin=vmin, vmax=vmax)

vmin = min(torch.min(old_linear_bias_data), torch.min(linear.bias.data))
vmax = max(torch.max(old_linear_bias_data), torch.max(linear.bias.data))

log_tensor(old_linear_bias_data, 'old_linear_weight_data', [class_labels], vmin=vmin, vmax=vmax)
log_tensor(linear.bias.data.detach(), 'linear.bias.data', [class_labels], vmin=vmin, vmax=vmax)